In [1]:
{-# LANGUAGE LambdaCase #-}

import qualified Data.Graph          as G
import qualified Data.HashMap.Strict as H
import qualified Data.HashSet        as S
import qualified Data.Array          as A

import Prelude hiding (lookup)

import Control.Monad.ST
import Data.STRef
import Control.Monad (forM_, when)
import Data.Maybe (isNothing, fromJust)

import Debug.Trace

denormalise = subtract
normalise   = (+)
other n v = 2*n - v
clauses n [u,v] = [(other n u, v), (other n v, u)]

tarjan :: Int -> G.Graph -> [S.HashSet Int]
tarjan n graph = runST $ do
    index    <- newSTRef (0 :: Int)
    stack    <- newSTRef []
    stackSet <- newSTRef S.empty
    indices  <- newSTRef H.empty
    lowlinks <- newSTRef H.empty
    output   <- newSTRef []

    forM_ (G.vertices graph) $ \v -> do
        vIndex <- H.lookup v <$> readSTRef indices
        when (isNothing vIndex) $
            strongConnect n v graph index stack stackSet indices lowlinks output

    readSTRef output


strongConnect
    :: (Num a, Ord a, Show a)
    => Int
    -> Int
    -> G.Graph
    -> STRef s a
    -> STRef s [Int]
    -> STRef s (S.HashSet Int)
    -> STRef s (H.HashMap Int a)
    -> STRef s (H.HashMap Int a)
    -> STRef s [S.HashSet Int]
    -> ST    s ()
strongConnect n v graph index stack stackSet indices lowlinks output = do
    i <- readSTRef index
    insert v i indices
    insert v i lowlinks
    increment index
    push stack stackSet v

    forM_ (graph A.! v) $ \w -> lookup w indices >>= \case
        Nothing     -> do
            strongConnect n w graph index stack stackSet indices lowlinks output
            vLowLink <- fromJust <$> lookup v lowlinks
            wLowLink <- fromJust <$> lookup w lowlinks
            insert v (min vLowLink wLowLink) lowlinks
        Just wIndex -> do
            wOnStack <- S.member w <$> readSTRef stackSet
            when wOnStack $ do
                vLowLink <- fromJust <$> lookup v lowlinks
                insert v (min vLowLink wIndex) lowlinks

    vLowLink <- fromJust <$> lookup v lowlinks
    vIndex   <- fromJust <$> lookup v indices
    when (vLowLink == vIndex) $ do
        scc <- addSCC n v S.empty stack stackSet
        modifySTRef' output (scc:)
    where
        lookup value hashMap     = H.lookup value <$> readSTRef hashMap
        insert key value hashMap = modifySTRef' hashMap (H.insert key value)
        increment counter        = modifySTRef' counter (+1)

addSCC :: Int -> Int -> S.HashSet Int -> STRef s [Int] -> STRef s (S.HashSet Int) -> ST s (S.HashSet Int)
addSCC n v scc stack stackSet = pop stack stackSet >>= \w -> if ((other n w) `S.member` scc) then error "not satisfiable" else
    let scc' = S.insert w scc
    in if w == v then return scc' else addSCC n v scc' stack stackSet

push :: STRef s [Int] -> STRef s (S.HashSet Int) -> Int -> ST s ()
push stack stackSet e = do
    modifySTRef' stack    (e:)
    modifySTRef' stackSet (S.insert e)

pop :: STRef s [Int] -> STRef s (S.HashSet Int) -> ST s Int
pop stack stackSet = do
    e <- head <$> readSTRef stack
    modifySTRef' stack tail
    modifySTRef' stackSet (S.delete e)
    return e

In [2]:
import qualified Data.ByteString as B
import qualified Data.ByteString.Char8 as BC
import Data.Maybe

In [3]:
checkSat name = do
    p <- map (map (fst . fromJust . BC.readInt) . BC.words) . BC.lines <$> B.readFile name
    let pNo    = head $ head p
        pn     = map (map (normalise pNo)) $ tail p
        pGraph = G.buildG (0,2*pNo) $ concatMap (clauses pNo) pn
    return $ length $ tarjan pNo pGraph

In [4]:
checkSat "2sat1.txt"

199991

In [5]:
checkSat "2sat2.txt"

not satisfiable
CallStack (from HasCallStack):
  error, called at <interactive>:61:99 in interactive:Ghci2

In [6]:
checkSat "2sat3.txt"

799667

In [7]:
checkSat "2sat4.txt"

1199985

In [8]:
checkSat "2sat5.txt"

not satisfiable
CallStack (from HasCallStack):
  error, called at <interactive>:61:99 in interactive:Ghci2

In [9]:
checkSat "2sat6.txt"

not satisfiable
CallStack (from HasCallStack):
  error, called at <interactive>:61:99 in interactive:Ghci2